In [16]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

In [17]:
# Función para extraer los datos de una URL
def extraer_datos_url(url):
    # Configuración del navegador
    browser.get(url)
    sleep(2)  # Esperar que cargue la página
    
    # Extraer nombre (título)
    try:
        nombre = browser.find_element(By.XPATH, "//h1").text
    except:
        nombre = None

    # Extraer precio
    try:
        precio = browser.find_element(By.CSS_SELECTOR, ".price__value.jsPriceValue").text
    except:
        precio = None

    return {"URL": url, "Nombre": nombre, "Precio": precio}

# Configuración del navegador
browser = webdriver.Chrome()
browser.get("https://www.pisos.com/")
browser.maximize_window()

# Aceptar cookies si es necesario
browser.find_element(by=By.CSS_SELECTOR, value="#didomi-notice-disagree-button").click()
sleep(1)

# Buscar "Madrid"
buscador = browser.find_element(by=By.XPATH, value='/html/body/main/div[1]/div/div[2]/div/div[2]/div/div[1]/input')
buscador.click()
sleep(1)
buscador.send_keys("Madrid")
sleep(1)

# Hacer clic en "Provincia"
browser.find_element(by=By.XPATH, value="//div[@class='location-item__subtitle' and text()='Provincia']").click()
sleep(1)

# Almacenar la URL actual como provincia_madrid
provincia_madrid = browser.current_url
print(provincia_madrid)  # Imprime el URL actual

# Inicializar la lista para almacenar los resultados
datos = []

# Acceder a las diferentes zonas y extraer las URLs
browser.find_element(by=By.CSS_SELECTOR, value="#main > div.inner-body.with-footer > div.content.subHome.subhome__wrapper > div:nth-child(5) > div.subhome__content > div:nth-child(1) > div.zoneList > nav > div > ul > li:nth-child(1) > a.seo-box__location-link.seo-box__location-link--level2").click()
sleep(1)

# Comarca1
comarca1_url = browser.current_url
print(f"Comarca1:{comarca1_url} ")

# Municipio1 Comarca1
browser.find_element(by=By.CSS_SELECTOR, value="#main > div.inner-body.with-footer > div.content.subHome.subhome__wrapper > div:nth-child(5) > div.subhome__content > div:nth-child(1) > div.zoneList > nav > div > ul > li:nth-child(1) > a.seo-box__location-link.seo-box__location-link--level2").click()
sleep(1)

municipio1_comarca1_url = browser.current_url
print(f"Municipio1 Comarca1:{municipio1_comarca1_url} ")

# Recoger la URL de los anuncios
browser.find_element(by=By.XPATH, value="//*[@id=\"5244455514.109700\"]").click()
sleep(1)

# Extraer datos de la URL actual
datos.append(extraer_datos_url(browser.current_url))

# Volver atrás y seguir con el siguiente
browser.back()
browser.find_element(by=By.XPATH, value="//*[@id=\"5227104703.109700\"]").click()
sleep(1)

# Extraer datos de la siguiente URL
datos.append(extraer_datos_url(browser.current_url))

# Convertir los resultados en un DataFrame de Pandas
df = pd.DataFrame(datos)

# Mostrar el DataFrame
print(df)

# Cerrar el navegador
browser.quit()

https://www.pisos.com/viviendas/madrid/
Comarca1:https://www.pisos.com/viviendas/corredor_del_henares/ 
Municipio1 Comarca1:https://www.pisos.com/venta/pisos-ajalvir/ 
                                                 URL  \
0  https://www.pisos.com/comprar/atico-ajalvir_ce...   
1  https://www.pisos.com/comprar/atico-ajalvir_ce...   

                              Nombre     Precio  
0  Ático en venta en Calle Real, s/n  329.700 €  
1  Ático en venta en Calle Real, s/n  177.700 €  


In [18]:
df.head()

,URL,Nombre,Precio
0,https://www.pisos.com/comprar/atico-ajalvir_ce...,"Ático en venta en Calle Real, s/n",329.700 €
1,https://www.pisos.com/comprar/atico-ajalvir_ce...,"Ático en venta en Calle Real, s/n",177.700 €
